In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/contradictory-my-dear-watson/sample_submission.csv
/kaggle/input/contradictory-my-dear-watson/train.csv
/kaggle/input/contradictory-my-dear-watson/test.csv


In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7")
model = AutoModelForSequenceClassification.from_pretrained("MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7")

In [21]:
from datasets import Dataset, load_dataset, Value

train_data = pd.read_csv("/kaggle/input/contradictory-my-dear-watson/train.csv").loc[:, ["premise", "hypothesis", "label"]]
test_data = pd.read_csv("/kaggle/input/contradictory-my-dear-watson/test.csv").loc[:, ["premise", "hypothesis", "id"]]

train_ds = Dataset.from_pandas(train_data)
test_ds = Dataset.from_pandas(test_data)

/opt/conda/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


In [4]:
def tokenize_function(examples):
    return tokenizer(examples["premise"], examples["hypothesis"], padding="max_length", truncation=True)
    
    return examples

In [5]:
tok_train_ds = train_ds.map(tokenize_function, batched=True).remove_columns(["premise", "hypothesis"])
tok_test_ds = test_ds.map(tokenize_function, batched=True).remove_columns(["premise", "hypothesis"])

  0%|          | 0/13 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

In [57]:
!pip install peft

In [10]:
from transformers import Trainer, TrainingArguments
from peft import IA3Model, IA3Config, TaskType


training_args = TrainingArguments(
    num_train_epochs=3,
    learning_rate=2e-05,
    gradient_accumulation_steps=2, 
    per_device_train_batch_size=8,
    warmup_ratio=0.06,               
    weight_decay=0.01,               
    fp16=False,
    output_dir="results",
    remove_unused_columns=False
)

peft_config = IA3Config(
    task_type=TaskType.CAUSAL_LM,
    target_modules=["key_proj", "value_proj", "down_proj"],
    feedforward_modules=["down_proj"]
)

ia3model = IA3Model(model=model, config=peft_config, adapter_name="default")
    
trainer = Trainer(
    model=ia3model,
    args=training_args,
    train_dataset=tok_train_ds)

trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
500,0.423900
1000,0.411200


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


TrainOutput(global_step=1137, training_loss=0.4212650913047287, metrics={'train_runtime': 2391.8242, 'train_samples_per_second': 15.202, 'train_steps_per_second': 0.475, 'total_flos': 9569034252656640.0, 'train_loss': 0.4212650913047287, 'epoch': 3.0})

In [11]:
trainer.save_model("./first_model")

In [56]:
from transformers import pipeline
from transformers.pipelines.pt_utils import KeyDataset

pipe = pipeline(task="sentiment-analysis", model=model, tokenizer=tokenizer, device="cuda")
ids = test_ds["id"]

label2num = {
    "entailment": 0,
    "neutral": 1,
    "contradiction": 2
}

with open("submission.csv", "w") as f:
    f.write("id,prediction\n")
    
with open("submission.csv", "a") as f:    
    for id_, out in zip(ids, pipe(KeyDataset(test_ds, "premise"))):
        f.write(id_ + ',' + str(label2num[out["label"]]) + "\n")
   